In [2]:
%%writefile psum.py
from openmpi.mpi4py import MPI
import numpy as np

def psum(a):
    locsum = np.sum(a)
    rcvBuf = np.array(0.0,'d')
    MPI.COMM_WORLD.Allreduce([locsum, MPI.DOUBLE],
        [rcvBuf, MPI.DOUBLE],
        op=MPI.SUM)
    return rcvBuf



Overwriting psum.py


In [3]:
from ipyparallel import Client
from openmpi.mpi4py import MPI
import numpy as np

ImportError: No module named 'openmpi'

In [4]:
c = Client(profile='default')

In [5]:
view = c[:]
print (view)

<DirectView [0, 1, 2, 3]>


In [6]:
view.activate()

In [7]:
view.run('psum.py')

<AsyncResult: execute>

In [8]:
view.scatter('a',np.arange(200,dtype='float'))

NameError: name 'np' is not defined

In [9]:
view['a']

CompositeError: one or more exceptions from call to method: _pull
[0:apply]: NameError: name 'a' is not defined
[1:apply]: NameError: name 'a' is not defined
[2:apply]: NameError: name 'a' is not defined
[3:apply]: NameError: name 'a' is not defined

In [10]:
%px totalsum = psum(a)

<AsyncResult: execute>

In [11]:
r = view['totalsum']

CompositeError: one or more exceptions from call to method: _pull
[0:apply]: NameError: name 'totalsum' is not defined
[1:apply]: NameError: name 'totalsum' is not defined
[2:apply]: NameError: name 'totalsum' is not defined
[3:apply]: NameError: name 'totalsum' is not defined

In [12]:
print(r[0])

NameError: name 'r' is not defined

In [13]:
print(r)

NameError: name 'r' is not defined

In [14]:
%%writefile hello.py
from mpi4py import MPI
from mpi4py.MPI import ANY_SOURCE
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
threads = comm.Get_size()
data = np.zeros(1)
if rank == 0:
    for i in range(threads):
        data[0] = i*100
        MPI.COMM_WORLD.Send(data, dest = i)
else:
    print(data[0])
    MPI.COMM_WORLD.Recv(data, source = 0)
    print(data[0])
    data[0] = data[0] + 4
    print("hello world from process ", rank, threads)
    
    
    MPI.COMM_WORLD.Send(data, dest = 0)
    
if rank == 0:
    for i in range(threads):
        MPI.COMM_WORLD.Recv(data, source = ANY_SOURCE)
        print(data[0])
        

Overwriting hello.py


In [15]:
!mpiexec  -n 4 python hello.py

Traceback (most recent call last):
  File "hello.py", line 1, in <module>
    from mpi4py import MPI
ImportError: No module named 'mpi4py'
Traceback (most recent call last):
  File "hello.py", line 1, in <module>
    from mpi4py import MPI
ImportError: No module named 'mpi4py'
-------------------------------------------------------
Primary job  terminated normally, but 1 process returned
a non-zero exit code.. Per user-direction, the job has been aborted.
-------------------------------------------------------
--------------------------------------------------------------------------
mpiexec detected that one or more processes exited with non-zero status, thus causing
the job to be terminated. The first process to do so was:

  Process name: [[55157,1],2]
  Exit code:    1
--------------------------------------------------------------------------


In [16]:
import numpy as np
from ipyparallel import Client

c = Client(profile='default')
dview = c[:]

A = np.random.random(64)

@dview.parallel(block=True)
def pmul(A,B):
    return (A*B)

C_local = A*A
C_remote = pmul(A,A)
(C_local == C_remote).all()

True

In [17]:
print (C_remote)

[  1.87076357e-01   4.34373718e-01   2.09051903e-01   6.23318488e-01
   1.12713715e-01   2.86212461e-02   2.06847790e-01   4.26171239e-01
   2.98113462e-01   3.88326309e-01   7.54830115e-01   3.03017118e-01
   6.86724083e-01   2.17988773e-01   3.07895229e-01   1.97824726e-03
   4.89140455e-01   3.67235061e-02   3.30963563e-01   1.08938258e-01
   5.72661764e-01   6.78652424e-01   3.88469769e-03   1.53369389e-03
   1.35032055e-01   3.77993894e-01   2.76475468e-01   4.75733698e-01
   1.89501533e-02   1.16722975e-01   1.74802841e-01   5.86090595e-02
   7.37626294e-01   3.25582256e-03   3.20608229e-02   6.69364568e-01
   5.18822543e-02   5.86515097e-01   4.36169758e-01   3.66280885e-01
   2.55226220e-01   1.06144228e-01   4.82653436e-01   8.03220749e-03
   9.19743350e-01   1.66383595e-01   3.61289646e-02   5.26521813e-02
   4.97225900e-02   4.85202158e-01   9.51160433e-01   2.69057310e-01
   3.13159807e-01   4.57318217e-02   3.98459608e-01   1.12658504e-01
   4.06252730e-01   1.20459917e-03

In [21]:
@dview.parallel(block=True)
def echo(x,y):
    return x,y

echo(range(8), range(4))

ValueError: all sequences must have equal length, but have [8, 4]

In [19]:
echo.map(range(8))

CompositeError: one or more exceptions from call to method: echo
[0:apply]: TypeError: echo() missing 1 required positional argument: 'y'
[1:apply]: TypeError: echo() missing 1 required positional argument: 'y'
[2:apply]: TypeError: echo() missing 1 required positional argument: 'y'
[3:apply]: TypeError: echo() missing 1 required positional argument: 'y'

In [20]:
a = range(100)
print(a[0:10])

range(0, 10)
